In [ ]:
import wikipedia

def wiki_suche(suchbegriff, sprache='de'):
    wikipedia.set_lang(sprache)

    try:
        # Direkten Artikel versuchen
        seite = wikipedia.page(suchbegriff)
        print(f"✅ Exakter Artikel gefunden: {seite.title}")
    except (wikipedia.DisambiguationError, wikipedia.PageError):
        # Kein direkter Artikel, versuche sinnvollen Alternativartikel
        vorschlaege = wikipedia.search(suchbegriff)
        for kandidat in vorschlaege:
            try:
                seite = wikipedia.page(kandidat)
                print(f"🔎 Automatisch weitergeleiteter Artikel: {seite.title}")
                print(f"🔗 URL: {seite.url}")
                return
            except:
                continue
        print("❌ Kein passender Artikel gefunden.")

# Beispiel
wiki_suche("FIFA-Korruptionsaffäre")


In [ ]:
import wikipedia
import requests

def wiki_suche(suchbegriff, sprache='de'):
    wikipedia.set_lang(sprache)

    try:
        # Direkten Artikel versuchen
        seite = wikipedia.page(suchbegriff)
        print(f"✅ Exakter Artikel gefunden: {seite.title}")
    except (wikipedia.DisambiguationError, wikipedia.PageError):
        # Kein direkter Artikel, versuche sinnvollen Alternativartikel
        vorschlaege = wikipedia.search(suchbegriff)
        for kandidat in vorschlaege:
            try:
                seite = wikipedia.page(kandidat)
                print(f"🔎 Automatisch weitergeleiteter Artikel: {seite.title}")
                print(f"🔗 URL: {seite.url}")
                break
            except:
                continue
        else:
            print("❌ Kein passender Artikel gefunden.")
            return

    print(f"🔗 URL: {seite.url}")
    print(f"\n🕘 Lade die komplette Bearbeitungshistorie für „{seite.title}“:\n")

    # Versionsgeschichte abrufen
    alle_revisionen(seite.title, sprache)

def alle_revisionen(titel, sprache='de'):
    revisions = []
    url = f"https://{sprache}.wikipedia.org/w/api.php"

    params = {
        "action": "query",
        "format": "json",
        "prop": "revisions",
        "titles": titel,
        "rvlimit": "500",  # max. erlaubt (für die erste Anfrage)
        "rvprop": "timestamp|user|comment",
        "redirects": 1
    }

    while True:
        response = requests.get(url, params=params)
        data = response.json()
        page = next(iter(data["query"]["pages"].values()))
        revs = page.get("revisions", [])
        revisions.extend(revs)

        # Fortschritt anzeigen
        print(f"↪️  {len(revisions)} Revisionen gesammelt...", end="\r")

        # Check auf weiteres "continue"-Token (Paginierung)
        if "continue" in data:
            params["rvcontinue"] = data["continue"]["rvcontinue"]
        else:
            break

    print(f"\n✅ Insgesamt {len(revisions)} Revisionen gefunden.\n")

    # Ausgabe der gesammelten Revisionen
    for rev in revisions:
        zeit = rev['timestamp'].replace("T", " ").replace("Z", "")
        benutzer = rev.get('user', 'Unbekannt')
        kommentar = rev.get('comment', '')
        print(f"- {zeit} | {benutzer} | {kommentar}")

    return revisions

# Beispiel
wiki_suche("FIFA-Korruptionsaffäre")
